# 03 - Battery Stress Detection Model Training

This notebook trains a neural network model to detect battery stress conditions. The model learns to identify when a battery is under stress based on parameters like current, temperature, voltage, and state of charge.


## Install OpenVINO Development Tools

Install OpenVINO toolkit for model optimization and conversion to Intel's Intermediate Representation (IR) format.


In [1]:
!pip install openvino-dev

## Disable GPU Usage

Configure TensorFlow to use CPU only, ensuring consistent execution across different environments.


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## Import Required Libraries

Import all necessary libraries for data processing, model building, and training.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow import keras
import joblib
import os
import subprocess
import shutil
import json

2025-11-20 20:53:42.212839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/app-root/lib64/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/app-root/lib64/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.

## Load Prepared Battery Data

Read the prepared battery data from the CSV file created in the previous notebook.


In [4]:
df = pd.read_csv("./data/battery_data.csv")

## Create Stress Indicator Labels

Define what constitutes a "stressed" battery based on threshold values for current, temperature, state of charge, and voltage. Create binary labels (0=normal, 1=stressed) for model training.


In [5]:
def detect_stress(row):
    if row["batteryCurrent"] > 400 or row["batteryTemp"] > 50 or row["stateOfCharge"] < 0.05 or row["batteryVoltage"] < 320:
        return 1
    return 0

df["stressIndicator"] = df.apply(detect_stress, axis=1)

## Select Features and Target Variable

Choose the input features (sensor readings) and the target variable (stress indicator) for the model.


In [6]:
features = ["stateOfCharge", "stateOfHealth", "batteryCurrent", "batteryVoltage", "kmh", "distance", "batteryTemp", "ambientTemp", "currentLoad"]
X = df[features]
y = df["stressIndicator"]

## Split and Normalize Data

Split the data into training and testing sets, then normalize the features using StandardScaler to improve model performance.


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = np.asarray(X_train_scaled, dtype=np.float32)
X_test_scaled = np.asarray(X_test_scaled, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.float32)
y_test = np.asarray(y_test, dtype=np.float32)

## Build Neural Network Model

Create a Multi-Layer Perceptron (MLP) with two hidden layers for binary classification of battery stress.


In [8]:
mlp_tf = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

mlp_tf.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

2025-11-20 20:54:49.869269: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## Train the Model

Train the neural network on the battery data for 50 epochs, using 10% of training data for validation.


In [9]:
mlp_tf.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9630 - loss: 0.4100 - val_accuracy: 1.0000 - val_loss: 0.2146
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.1313 - val_accuracy: 1.0000 - val_loss: 0.0599
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0415 - val_accuracy: 1.0000 - val_loss: 0.0220
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0182 - val_accuracy: 1.0000 - val_loss: 0.0113
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0104 - val_accuracy: 1.0000 - val_loss: 0.0069
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0067 - val_accuracy: 1.0000 - val_loss: 0.0047
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 1.0000 - val_loss:

## Save Model and Scaler

Save the trained model and the data scaler for later use in prediction.


In [10]:
os.makedirs("models", exist_ok=True)
scaler_path = "models/stress_scaler.pkl"
joblib.dump(scaler, scaler_path)
mlp_tf.save("models/battery_stress_model.keras")

## Export Model to TensorFlow SavedModel Format

Export the model in TensorFlow's SavedModel format, which is needed for conversion to OpenVINO IR.


In [11]:
try:
    mlp_tf.export("models/battery_stress_model")
except AttributeError:
    mlp_tf.save("models/battery_stress_model", save_format='tf')

INFO:tensorflow:Assets written to: models/battery_stress_model/assets


INFO:tensorflow:Assets written to: models/battery_stress_model/assets


Saved artifact at 'models/battery_stress_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139974756563856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139974756564816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139974756563664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139974756563280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139974756563088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139974756562128: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Convert Model to OpenVINO IR Format

Convert the TensorFlow model to OpenVINO Intermediate Representation (IR) for optimized inference on Intel hardware.


In [14]:
os.makedirs("models/battery_stress_model_ir", exist_ok=True)

subprocess.run(
    [
        "ovc",
        "models/battery_stress_model",
        "--output_model", "models/battery_stress_model_ir/saved_model"
    ],
    capture_output=True,
    text=True,
    check=True
)

CompletedProcess(args=['ovc', 'models/battery_stress_model', '--output_model', 'models/battery_stress_model_ir/saved_model'], returncode=0, stdout='[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression by removing argument "compress_to_fp16" or set it to false "compress_to_fp16=False".\nFind more information about compression to FP16 at https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_FP16_Compression.html\n[ SUCCESS ] XML file: models/battery_stress_model_ir/saved_model.xml\n[ SUCCESS ] BIN file: models/battery_stress_model_ir/saved_model.bin\n', stderr='')

## Prepare Model for Serving

Organize the IR model files into a directory structure suitable for serving with OpenVINO Model Server.


In [13]:

def prepare_ir_model_for_serving(ir_model_path, output_path, version=1):
    version_path = os.path.join(output_path, str(version))
    os.makedirs(version_path, exist_ok=True)
    
    for item in os.listdir(ir_model_path):
        if item.endswith(('.xml', '.bin')):
            src = os.path.join(ir_model_path, item)
            dst = os.path.join(version_path, item)
            shutil.copy2(src, dst)
    
    return version_path

serving_path = "models/serving/battery_stress_model"
ir_model_path = "models/battery_stress_model_ir"

model_version_path = prepare_ir_model_for_serving(ir_model_path, serving_path, version=1)